In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib

# Carregar os dados
df = pd.read_csv("Data/DataFramelimpa_sem_latlong.csv")

# Pré-processamento dos Dados
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

# Função para extrair características de data/hora
def extract_datetime_features(df):
    df['purchase_weekday'] = df['order_purchase_timestamp'].dt.weekday
    df['purchase_month'] = df['order_purchase_timestamp'].dt.month
    df['purchase_hour'] = df['order_purchase_timestamp'].dt.hour
    df['approval_delay'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600  # em horas
    return df

# Calcular o volume do produto
df['product_volume'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']

# Adicionar a variável de frete (freight_value)
df['freight_value'] = df['freight_value']

df = extract_datetime_features(df)

# Preencher valores ausentes com a mediana das colunas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Variáveis preditoras e alvo, incluindo order_id e order_purchase_timestamp para uso futuro
X = df[['order_id', 'order_purchase_timestamp', 'delivery_time_model', 'purchase_weekday', 'purchase_month', 'purchase_hour', 'approval_delay', 'product_volume', 'product_weight_g', 'seller_zip_code_prefix', 'customer_zip_code_prefix', 'freight_value', 'price']]
y = df['delivery_time']  # Usar delivery_time já existente

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline de pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['approval_delay', 'product_volume', 'product_weight_g', 'freight_value', 'price']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['purchase_weekday', 'purchase_month', 'purchase_hour', 'seller_zip_code_prefix', 'customer_zip_code_prefix'])
    ])

# Pipeline completo com regressor RandomForestRegressor
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42, n_jobs=-1))
])

# Definir a grade de hiperparâmetros
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4],
    'regressor__bootstrap': [True, False]
}

# GridSearchCV para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(estimator=model_pipeline, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')

# Treinar o modelo com GridSearch
grid_search.fit(X_train.drop(columns=['order_id', 'order_purchase_timestamp', 'delivery_time_model']), y_train)

# Melhor modelo encontrado
best_model = grid_search.best_estimator_

# Fazer previsões nos dados de teste
y_pred = best_model.predict(X_test.drop(columns=['order_id', 'order_purchase_timestamp', 'delivery_time_model']))

# Diagnóstico para verificar as previsões
print("Valores de previsão extremos:", y_pred[(y_pred < 0) | (y_pred > 365)]) 

# Limitar valores de previsão a um intervalo razoável, por exemplo, 0 a 365 dias (1 ano)
y_pred = np.clip(y_pred, 0, 365)

# Criar DataFrame com as previsões e os resultados reais
df_resultado = pd.DataFrame({
    'OrderID': X_test['order_id'].values,
    'Delivery Time Real': y_test,
    'Delivery Time Previsto (Antigo)': X_test['delivery_time_model'],
    'Delivery Time Previsto (Novo)': y_pred,
})

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'Melhores hiperparâmetros: {grid_search.best_params_}')

# Salvar o modelo para uso futuro
joblib.dump(best_model, 'best_model_pipeline_tunado.pkl')

# Salvar resultado como CSV
df_resultado.to_csv('Data/resultado_previsoes.csv', index=False)

print("CSV de previsões salvo com sucesso!")
